In [2]:
import os
from datetime import datetime
import yaml
from IPython.display import display, Markdown
from git import Repo as RepoLoader  # gitpython

from tqdm import tqdm

import papers_analyser.github as github
import papers_analyser.db as db
from papers_analyser.paper import load_papers,download_json_files
import pandas as pd

%load_ext autoreload
%autoreload 2

In [3]:
project_dir = os.path.abspath("..")

## Connect to DB

In [4]:
connection = db.create_connection( project_dir + "/data/paper.db")
db.create_table_if_not_exist(connection)

2.6.0


## Github API auth-token

In [5]:
auth_token = github.load_auth_token("config.yaml")

C:\Users\Peter-NB\Dev\reproduce-ml-papers\papers_analyser\github.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open(path, "r"))


# Loading papers

In [6]:
json_path = project_dir + "/data"

Download JSON file if you have not done this already.

In [6]:
download_json_files(json_path)

Load json files and create paper objects that map to repositories.

In [7]:
since_date = datetime.strptime("2019-01-01", '%Y-%m-%d')


In [8]:
papers= load_papers(project_dir + "/data",  since_date)


In [13]:
papers_persisted = db.get_papers(connection)

In [14]:
papers_to_scrape = [paper for paper in papers if paper.url not in papers_persisted]

Scrape the metadata, file-names and the whole readme-file from the github-api for each repo.
If you have a lot of repos to scrape, you have to rerunt his cell multiple times.

In [22]:
for paper in tqdm(papers_to_scrape[30:]):
    for repo in paper.repos:
        if True:#not repo.scraped:
            repo.scrape(auth_token)

100%|██████████| 62/62 [07:09<00:00,  6.92s/it]


404 Client Error: Not Found for url: https://api.github.com/repos/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/classifiers/rusentiment_bert.json
404 Client Error: Not Found for url: https://api.github.com/repos/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/classifiers/rusentiment_convers_bert.json
404 Client Error: Not Found for url: https://api.github.com/repos/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/classifiers/rusentiment_elmo_twitter_cnn.json
404 Client Error: Not Found for url: https://api.github.com/repos/NeurEXT/NEXT-learning-to-plan/blob/master/main.ipynb
404 Client Error: Not Found for url: https://api.github.com/repos/PKU-AI-Edge/DGN/
404 Client Error: Not Found for url: https://api.github.com/repositories/258255951/blob/master/examples/iMAML.py
404 Client Error: Not Found for url: https://api.github.com/repos/yanmeizhao/MyMmdetection/branches/master
404 Client Error: Not Found for url: https://api.github.com/repos/Tony607/mmdetection/branches/master
404

## Take a look on README files

In [22]:
readmes = [paper.repo[0].readme for paper in papers]

NameError: name 'papers' is not defined

In [36]:
display(Markdown(readmes[4]))

NameError: name 'readmes' is not defined

# Save to database

In [23]:
i=1
j=1
unscraped = list()
for paper in papers_to_scrape:
    if paper.repos_scraped():
        db.insert_paper(connection,paper)
    else:
        unscraped.append(paper)

connection.commit()

The number of paprs containing a repo that was not able to scrape is:

In [25]:
len(unscraped)


24

In [ ]:
len(papers_to_scrape)

In [26]:
connection.close()

## Clone repo

In [36]:
dir = driverPath = project_dir + "/repos"
for paper in papers:
    paper_path = dir + "/" + paper.title.replace( ":","")
    if not os.path.exists(paper_path):
        os.mkdir(paper_path)
        
    for repo in paper.repo:
        repo_path = paper_path+ repo.repo_name.replace("/","_")
        
        if not os.path.exists(repo_path):
            os.mkdir(repo_path)
            
        RepoLoader.clone_from(repo.clone_url,repo_path)
    

